In [9]:
import json
import requests


video = {
    # "path": "/workspaces/aana_sdk/aana/tests/files/videos/physicsworks.webm"
    # "url": "https://www.youtube.com/watch?v=yModCU1OVHY",
    # "media_id": "yModCU1OVHY",
    # "url": "https://www.youtube.com/watch?v=77ZF50ve6rs",
    # "media_id": "77ZF50ve6rs",
    "url": "https://www.youtube.com/watch?v=wxN1T1uxQ2g",
    "media_id": "wxN1T1uxQ2g",
}

In [10]:
data = {
    "video_params": {
        "fast_mode_enabled": True,
    },
    "video": video,
}
url = "http://127.0.0.1:8000/video/index_stream"
response = requests.post(url, data={"body": json.dumps(data)}, stream=True)
# visuals = response.json()
for chunk in response.iter_content(chunk_size=None):
    print(chunk)

b'{"timestamps":[0.0,3.003000020980835,6.631625175476074,8.55020809173584,10.927583694458008,12.512499809265137,14.013999938964844,16.80845832824707],"captions":["a close up of a circle with a circle in the center","a picture of a person on a skateboard","a woman in an office setting with papers and papers","three people sitting at a desk with papers and papers","a2a logo design","a woman standing in front of a desk full of papers","a woman sitting at a table with a laptop and a tv","a woman and two men standing in front of a washing machine"]}'
b'{"transcription":{"text":" This is Wang. This is Wang. Mrs. Wang, are you with us? I am paying attention. Now you may only see a pile of receipts, but I see a story. I can see where this story is going. It does not look good. What\'s happening? Evelyn, I\'m not your husband. I\'m another version of Wang from another universe. I\'m here because we need your help. I\'m very busy today. I don\'t have time to help you. Across the multiverse, I\'v

In [11]:
media_id = video["media_id"]

question = (
    "What is happening in this video? "
    "Focus on highlighting the story, the narrative, emotionally and thematically significant aspects. "
    "Do not use bulleted list in your answer."
)

In [12]:
data = {
    "question": question,
    "media_id": media_id,
    "sampling_params": {
        "temperature": 0.9,
        "max_tokens": 1024,
    },
}

url = "http://127.0.0.1:8000/video/chat_stream"

response = requests.post(url, data={"body": json.dumps(data)}, stream=True)
text = ""
for chunk in response.iter_content(chunk_size=None):
    chunk_dict = json.loads(chunk)
    if "error" in chunk_dict:
        print(chunk_dict["error"])
        print(chunk_dict["message"])
        print(chunk_dict["stacktrace"])
        break
    token = chunk_dict["completion"]
    text += token
    print(token, end="")

  Based on the provided video script, I can help you answer your question. The video presents a narrative that revolves around the theme of multiverse and the concept of "Evelyn," who is a version of Wang from another universe. The story follows Wang as she navigates through different realities, meeting different versions of herself and encountering various obstacles.
Here are the most relevant and aesthetically striking moments in the video:
1. The opening scene shows a close-up of a circle with a circle in the center, followed by a picture of a person on a skateboard, a woman in an office setting with papers and papers, and three people sitting at a desk with papers and papers. These images represent the first few moments of the video, where Wang is introduced as a character who is trying to understand the concept of multiverse.
2. The visual caption for the 20-25 time frame shows a woman in a green cloak standing in the woods, which suggests a moment of serenity and contemplation fo